In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from acv_explainers.utils import *
from ares import AReS
from cet import CounterfactualExplanationTree
from ce import ActionExtractor
from utils import DatasetHelper, submodular_picking, DATASETS_NAME
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

seed = 2022
np.random.seed(0)

In [2]:
# Load breast cancer dataset

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.20, random_state=seed)

In [3]:
# Train the Isolation for outlier score

from sklearn.ensemble import IsolationForest

isolation = IsolationForest()
isolation.fit(X_tr)

IsolationForest()

In [5]:
y_tr = 1 - y_tr
y_ts = 1 - y_ts

In [7]:
# Train the model to explain

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

LGBMClassifier(n_estimators=50, num_leaves=8)

In [8]:
feature_names = [col.replace(' ', '') for col in X_tr.columns]
feature_names

['meanradius',
 'meantexture',
 'meanperimeter',
 'meanarea',
 'meansmoothness',
 'meancompactness',
 'meanconcavity',
 'meanconcavepoints',
 'meansymmetry',
 'meanfractaldimension',
 'radiuserror',
 'textureerror',
 'perimetererror',
 'areaerror',
 'smoothnesserror',
 'compactnesserror',
 'concavityerror',
 'concavepointserror',
 'symmetryerror',
 'fractaldimensionerror',
 'worstradius',
 'worsttexture',
 'worstperimeter',
 'worstarea',
 'worstsmoothness',
 'worstcompactness',
 'worstconcavity',
 'worstconcavepoints',
 'worstsymmetry',
 'worstfractaldimension']

In [26]:
target_name = 'Cancer'
target_labels = ['Good', 'Bad']

In [10]:
feature_constraints = ['' for i in range(len(feature_names))]
len(feature_constraints) == X_tr.shape[1]

True

In [11]:
feature_categories = []

In [12]:
feature_types = ['C' for i in range(len(feature_names))]

In [13]:
X_tr = X_tr.values
X_ts = X_ts.values

In [14]:
y_tr = y_tr.values
y_ts = y_ts.values

In [15]:
X = X_tr[mdl.predict(X_tr)==1]
X_vl = X_ts[mdl.predict(X_ts)==1]

## Actionable Recourse Summary

In [18]:
print('## Actionable Recourse Summary')
ares = AReS(mdl, X_tr, max_rule=4, max_rule_length=4, discretization_bins=10, minimum_support=0.05, print_objective=False,
            feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories, 
            feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)
ares = ares.fit(X, max_change_num=4, cost_type='MPS', lambda_acc=1.0, lambda_cov=1.0, lambda_cst=1.0)
print('* Parameters:')
print('\t* lambda_acc: {}'.format(ares.lambda_acc)); print('\t* lambda_cov: {}'.format(ares.lambda_cov)); print('\t* lambda_cst: {}'.format(ares.lambda_cst));
print('\t* minimum support: {}'.format(ares.rule_miner_.minsup_)); print('\t* discretization bins: {}'.format(ares.rule_miner_.fd_.bins)); print('\t* pre-processing time[s]: {}'.format(ares.preprocess_time_)); 
print('\t* max rule: {}'.format(ares.max_rule_)); print('\t* max rule length: {}'.format(ares.max_rule_length_)); print('\t* Time[s]:', ares.time_); 
print('\t* uncover test: {}'.format(ares.uncover(X_vl))); print('\t* conflict: {}'.format(ares.conflict(X_vl))); print();
print('### Learned AReS')
print(ares.to_markdown())

## Actionable Recourse Summary
* Parameters:
	* lambda_acc: 1.0
	* lambda_cov: 1.0
	* lambda_cst: 1.0
	* minimum support: 0.05
	* discretization bins: 10
	* pre-processing time[s]: 0.44749303099979443
	* max rule: 4
	* max rule length: 4
	* Time[s]: 179.7428058229998
	* uncover test: 0.41304347826086957
	* conflict: 0.0

### Learned AReS
| | Rule | Action |
| :---: | --- | --- |
| Recourse <br> rule 1 <br> (probability: 16.9%) | If '17.74<=x_20<20.4' <br> AND '118.1<=x_22<136.6' <br> AND '971.7<=x_23<1.293e+03' | 13.65<=x_20<14.48 <br> AND 88.71<=x_22<95.11 <br> AND 568.2<=x_23<637.7 |
| Recourse <br> rule 2 <br> (probability: 6.6%) | If '16.34<=x_20<17.74' <br> AND '108.5<=x_22<118.1' <br> AND '812.3<=x_23<971.7' | 13.65<=x_20<14.48 <br> AND 88.71<=x_22<95.11 <br> AND 568.2<=x_23<637.7 |
| Recourse <br> rule 3 <br> (probability: 18.1%) | If '20.4<=x_20<23.93' <br> AND '136.6<=x_22<159.8' <br> AND '1.293e+03<=x_23<1.72e+03' | 12.36<=x_20<13.1 <br> AND 79.79<=x_22<84.89 <br> AND 469.0<=

In [27]:
x_ares = ares.predict(X_vl) + X_vl
y_ares = mdl.predict(x_ares)

print('Negative accuracy of AReS = {}'.format(np.mean(y_ares != mdl.predict(X_vl))))
print('Negative Inlier AReS = {}'.format(np.mean(isolation.predict(x_ares) == 1)))

Negative accuracy of AReS = 0.34782608695652173
Negative Inlier AReS = 0.8043478260869565


In [28]:
print('Sparsity', np.mean(np.sum(x_ares-X_vl!=0, axis=1)))

Sparsity 2.9565217391304346


# Counterfactual TREE

In [20]:
LAMBDA = 0.01
GAMMA = 1.0

print('## Counterfactual Explanation Tree')
cet = CounterfactualExplanationTree(mdl, X_tr, y_tr, max_iteration=500, lime_approximation=False,
                                    feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories,
                                    feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)

cet = cet.fit(X, max_change_num=8, cost_type='MPS', C=LAMBDA, gamma=GAMMA, max_leaf_size=-1, time_limit=180, verbose=True)
print('* Parameters:'); print('\t* lambda: {}'.format(cet.lambda_)); print('\t* gamma: {}'.format(cet.gamma_)); print('\t* max_iteration: {}'.format(cet.max_iteration_));
print('\t* leaf size bound:', cet.leaf_size_bound_); print('\t* leaf size:', cet.n_leaves_); print('\t* LIME approximation:', cet.lime_approximation_); print('\t* Time[s]:', cet.time_); print();
print('### Learned CET')
cet.print_tree()

## Counterfactual Explanation Tree
## Stochastic Local Searching ...
### Iteration: 10
#### Before:
- If smoothnesserror<0.007:
	- If worstradius<15.95:
		- If radiuserror<0.2717:
			* Action [Cancer: Bad -> Good] (4/4 = 100.0% / MeanCost = 0.401):
				* meantexture: -4.2630
				* meanconcavepoints: -0.0203
				* worstradius: -3.7268
				* worsttexture: -6.2286
				* worstperimeter: -23.9794
				* worstarea: -349.7480
				* worstconcavity: -0.2085
				* worstconcavepoints: -0.0705
		- Else:
			- If perimetererror<2.5722:
				* Action [Cancer: Bad -> Good] (3/3 = 100.0% / MeanCost = 0.621):
					* meanconcavepoints: -0.0565
					* concavityerror: +0.2626
					* worstradius: -7.2700
					* worsttexture: -11.2554
					* worstperimeter: -52.3900
					* worstarea: -520.8000
					* worstsmoothness: -0.0507
					* worstconcavepoints: -0.1337
			- Else:
				* Action [Cancer: Bad -> Good] (3/3 = 100.0% / MeanCost = 0.465):
					* meantexture: -5.4730
					* meanconcavepoints: -0.0250
					* wor

### Iteration: 30
#### Before:
- If smoothnesserror<0.007:
	- If meantexture<15.698:
		* Action [Cancer: Bad -> Good] (4/5 = 80.0% / MeanCost = 0.509):
			* meanconcavepoints: -0.0624
			* areaerror: -26.4680
			* worstradius: -7.3290
			* worsttexture: -5.3100
			* worstperimeter: -53.8110
			* worstarea: -714.3240
			* worstsmoothness: -0.0208
			* worstconcavepoints: -0.1049
	- Else:
		- If meantexture<22.416:
			* Action [Cancer: Bad -> Good] (0/71 = 0.0% / MeanCost = -6.74e-09):
		- Else:
			* Action [Cancer: Bad -> Good] (0/32 = 0.0% / MeanCost = 0.0):
- Else:
	- If fractaldimensionerror<0.002:
		* Action [Cancer: Bad -> Good] (3/3 = 100.0% / MeanCost = 0.571):
			* meantexture: -6.9830
			* meanconcavepoints: -0.0504
			* concavityerror: +0.2593
			* worstradius: -7.3424
			* worsttexture: -10.2766
			* worstperimeter: -50.2794
			* worstarea: -712.2960
			* worstconcavepoints: -0.1061
	- Else:
		- If worstsmoothness<0.1382:
			- If worstperimeter<251.2:
				* Action [Cancer: Ba

- If meansymmetry<0.172:
	- If meanperimeter<110.12:
		- If meanarea<607.54:
			* Action [Cancer: Bad -> Good] (5/5 = 100.0% / MeanCost = 0.425):
				* meanradius: -3.5113
				* meantexture: -4.8972
				* meanconcavepoints: -0.0270
				* worstradius: -4.3056
				* worsttexture: -8.1722
				* worstperimeter: -30.6742
				* worstarea: -368.4000
				* worstconcavepoints: -0.0716
		- Else:
			* Action [Cancer: Bad -> Good] (8/8 = 100.0% / MeanCost = 0.384):
				* meanradius: -3.0506
				* meantexture: -4.0130
				* meanconcavepoints: -0.0285
				* worstradius: -4.2490
				* worsttexture: -6.8844
				* worstperimeter: -29.0668
				* worstarea: -432.2240
				* worstconcavepoints: -0.0290
	- Else:
		* Action [Cancer: Bad -> Good] (0/22 = 0.0% / MeanCost = 0.0):
- Else:
	- If fractaldimensionerror<0.002:
		- If perimetererror<2.5722:
			* Action [Cancer: Bad -> Good] (5/5 = 100.0% / MeanCost = 0.445):
				* meanconcavepoints: -0.0203
				* worstradius: -4.6268
				* worsttexture: -6.5264
				*

### Iteration: 70
#### Before:
- If meansymmetry<0.172:
	- If meanarea<607.54:
		* Action [Cancer: Bad -> Good] (5/5 = 100.0% / MeanCost = 0.425):
			* meanradius: -3.5113
			* meantexture: -4.8972
			* meanconcavepoints: -0.0270
			* worstradius: -4.3056
			* worsttexture: -8.1722
			* worstperimeter: -30.6742
			* worstarea: -368.4000
			* worstconcavepoints: -0.0716
	- Else:
		* Action [Cancer: Bad -> Good] (0/30 = 0.0% / MeanCost = 0.0):
- Else:
	- If fractaldimensionerror<0.002:
		- If radiuserror<0.2717:
			* Action [Cancer: Bad -> Good] (5/5 = 100.0% / MeanCost = 0.445):
				* meanconcavepoints: -0.0203
				* worstradius: -4.6268
				* worsttexture: -6.5264
				* worstperimeter: -32.0110
				* worstarea: -392.7480
				* worstsmoothness: -0.0267
				* worstconcavity: -0.2085
				* worstconcavepoints: -0.0815
		- Else:
			- If worstsmoothness<0.1126:
				* Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.46):
					* meantexture: -4.9774
					* meanconcavepoints: -0.0399
	

### Iteration: 80
#### Before:
- If meansymmetry<0.172:
	- If meanarea<607.54:
		* Action [Cancer: Bad -> Good] (5/5 = 100.0% / MeanCost = 0.425):
			* meanradius: -3.5113
			* meantexture: -4.8972
			* meanconcavepoints: -0.0270
			* worstradius: -4.3056
			* worsttexture: -8.1722
			* worstperimeter: -30.6742
			* worstarea: -368.4000
			* worstconcavepoints: -0.0716
	- Else:
		* Action [Cancer: Bad -> Good] (0/30 = 0.0% / MeanCost = 0.0):
- Else:
	- If fractaldimensionerror<0.002:
		- If concavepointserror<0.0096:
			* Action [Cancer: Bad -> Good] (6/7 = 85.7% / MeanCost = 0.452):
				* meanradius: -3.6535
				* meanconcavepoints: -0.0203
				* worstradius: -5.1890
				* worsttexture: -7.1942
				* worstperimeter: -36.0268
				* worstarea: -474.1240
				* worstconcavity: -0.1547
				* worstconcavepoints: -0.0658
		- Else:
			* Action [Cancer: Bad -> Good] (0/4 = 0.0% / MeanCost = 0.0):
	- Else:
		- If meansymmetry<0.1851:
			- If worsttexture<20.418:
				* Action [Cancer: Bad -> Goo

...

- If worstfractaldimension<0.0697:
	* Action [Cancer: Bad -> Good] (0/21 = 0.0% / MeanCost = -3.04e-10):
- Else:
	- If radiuserror<0.3643:
		- If meanradius<12.712:
			* Action [Cancer: Bad -> Good] (4/4 = 100.0% / MeanCost = 0.32):
				* meanconcavepoints: -0.0325
				* worstradius: -2.6612
				* worsttexture: -5.7198
				* worstperimeter: -16.7310
				* worstarea: -241.5240
				* worstsmoothness: -0.0413
				* worstconcavity: -0.2270
				* worstconcavepoints: -0.0726
		- Else:
			* Action [Cancer: Bad -> Good] (30/31 = 96.8% / MeanCost = 0.48):
				* meantexture: -4.7986
				* meanconcavepoints: -0.0203
				* worstradius: -4.8734
				* worsttexture: -8.2200
				* worstperimeter: -37.6142
				* worstarea: -387.0240
				* worstconcavity: -0.2085
				* worstconcavepoints: -0.0874
	- Else:
		- If worstsymmetry<0.327:
			- If meanradius<14.054:
				* Action [Cancer: Bad -> Good] (4/4 = 100.0% / MeanCost = 0.607):
					* meanconcavepoints: -0.0449
					* worstradius: -6.6934
					* worstte

- If meanconcavity<0.024:
	* Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.365):
		* meanradius: -2.9380
		* meantexture: -6.0276
		* radiuserror: -0.8237
		* areaerror: -67.0741
		* worstradius: -3.1246
		* worsttexture: -5.8342
		* worstperimeter: -25.0510
		* worstarea: -188.0960
- Else:
	- If compactnesserror<0.0244:
		* Action [Cancer: Bad -> Good] (0/61 = 0.0% / MeanCost = 0.0):
	- Else:
		- If textureerror<1.556:
			- If areaerror<20.732:
				* Action [Cancer: Bad -> Good] (2/2 = 100.0% / MeanCost = 0.86):
					* meantexture: -9.1200
					* meanconcavepoints: -0.0637
					* concavityerror: +0.2518
					* worstradius: -5.3100
					* worsttexture: -19.3222
					* worstperimeter: -41.3500
					* worstarea: -322.9000
					* worstconcavepoints: -0.1859
			- Else:
				* Action [Cancer: Bad -> Good] (0/75 = 0.0% / MeanCost = 0.0):
		- Else:
			- If radiuserror<0.2143:
				* Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.317):
					* meanconcavepoints: -0.0332
			

- If meanperimeter<73.052:
	* Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.223):
		* meanconcavepoints: -0.0245
		* worstradius: -2.6612
		* worsttexture: -5.9572
		* worstperimeter: -12.7152
		* worstarea: -166.0480
		* worstsmoothness: -0.0440
		* worstconcavity: -0.1018
		* worstconcavepoints: -0.0493
- Else:
	- (+) If worstperimeter<131.28:
		* (+) Action [Cancer: Bad -> Good] (0/74 = 0.0% / MeanCost = -7.91e-09):
	- Else:
		- (+) If worstfractaldimension<0.0832:
			- (+) If concavepointserror<0.0156:
				- (+) If worstfractaldimension<0.077:
					- (+) If concavepointserror<0.0069:
						* (+) Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.536):
							* meantexture: -8.8700
							* meanconcavepoints: -0.0461
							* worstradius: -7.7580
							* worsttexture: -10.3332
							* worstperimeter: -52.6478
							* worstarea: -817.9200
							* worstconcavity: -0.2536
							* worstconcavepoints: -0.0989
					- Else:
						* (+) Action [Cancer: Bad -> Good] (

### Iteration: 460
#### Before:
- If worsttexture<45.41:
	* Action [Cancer: Bad -> Good] (0/165 = 0.0% / MeanCost = -3.46e-10):
- Else:
	* Action [Cancer: Bad -> Good] (0/1 = 0.0% / MeanCost = 0.0):

#### After "Replace"
- (+) If meanfractaldimension<0.05:
	* (+) Action [Cancer: Bad -> Good] (1/1 = 100.0% / MeanCost = 0.336):
		* meantexture: -3.1732
		* meanconcavepoints: -0.0343
		* worstradius: -5.2836
		* worsttexture: -5.6020
		* worstperimeter: -34.4162
		* worstarea: -562.5440
		* worstconcavity: -0.1459
		* worstconcavepoints: -0.0596
- Else:
	* (+) Action [Cancer: Bad -> Good] (0/165 = 0.0% / MeanCost = -5.02e-10):

##### Score:
- Objective: 1.0199999996558438 => 1.0159987122742338
- Update: True
- Best: False
- Time: 501.59728790700046

Ckeckpoint !
### Iteration: 470
#### Before:
- If worstconcavity<0.2889:
	* Action [Cancer: Bad -> Good] (0/29 = 0.0% / MeanCost = -2.32e-10):
- Else:
	* Action [Cancer: Bad -> Good] (0/137 = 0.0% / MeanCost = 0.0):

#### After "Insert"
- If w

- If worstfractaldimension<0.0832:
	- If worstconcavity<0.0906:
		* Action [Cancer: Bad -> Good] (2/2 = 100.0% / MeanCost = 0.38):
			* meanradius: -3.5113
			* meantexture: -3.6988
			* meanconcavepoints: -0.0209
			* areaerror: -20.8921
			* worstradius: -4.3056
			* worsttexture: -6.0276
			* worstperimeter: -34.6900
			* worstarea: -287.0240
	- Else:
		- If radiuserror<0.5368:
			* Action [Cancer: Bad -> Good] (29/31 = 93.5% / MeanCost = 0.499):
				* meantexture: -5.9572
				* meanconcavepoints: -0.0203
				* worstradius: -5.9978
				* worsttexture: -8.6644
				* worstperimeter: -41.6300
				* worstarea: -468.4000
				* worstconcavity: -0.1211
				* worstconcavepoints: -0.0658
		- Else:
			* Action [Cancer: Bad -> Good] (0/36 = 0.0% / MeanCost = 0.0):
- Else:
	- If meanconcavepoints<0.0838:
		- If concavepointserror<0.0156:
			* Action [Cancer: Bad -> Good] (0/29 = 0.0% / MeanCost = 0.0):
		- Else:
			* Action [Cancer: Bad -> Good] (6/6 = 100.0% / MeanCost = 0.673):
				* meantext

In [29]:
x_cet = cet.predict(X_vl) + X_vl
y_cet = mdl.predict(x_cet)

print('Negative accuracy of CET = {}'.format(np.mean(y_cet != mdl.predict(X_vl))))
print('Negative Inlier CET = {}'.format(np.mean(isolation.predict(x_cet) == 1)))

Negative accuracy of CET = 0.21739130434782608
Negative Inlier CET = 0.8260869565217391


In [30]:
print('Sparsity', np.mean(np.sum(x_cet-X_vl!=0, axis=1)))

Sparsity 1.9130434782608696


In [31]:
cet.print_tree()

- If worstfractaldimension<0.0832:
	- If worstconcavity<0.0906:
		* Action [Cancer: Bad -> Good] (2/2 = 100.0% / MeanCost = 0.38):
			* meanradius: -3.5113
			* meantexture: -3.6988
			* meanconcavepoints: -0.0209
			* areaerror: -20.8921
			* worstradius: -4.3056
			* worsttexture: -6.0276
			* worstperimeter: -34.6900
			* worstarea: -287.0240
	- Else:
		- If radiuserror<0.5368:
			* Action [Cancer: Bad -> Good] (29/31 = 93.5% / MeanCost = 0.499):
				* meantexture: -5.9572
				* meanconcavepoints: -0.0203
				* worstradius: -5.9978
				* worsttexture: -8.6644
				* worstperimeter: -41.6300
				* worstarea: -468.4000
				* worstconcavity: -0.1211
				* worstconcavepoints: -0.0658
		- Else:
			* Action [Cancer: Bad -> Good] (0/36 = 0.0% / MeanCost = 0.0):
- Else:
	- If meanarea<607.54:
		- If concavepointserror<0.0156:
			* Action [Cancer: Bad -> Good] (16/16 = 100.0% / MeanCost = 0.619):
				* meanconcavepoints: -0.0507
				* concavityerror: +0.2285
				* worstradius: -4.9100
				* w